# Creating a Vector DB

Before creating our vector search, we need to create a database of vectors. 


In [1]:
import iris
conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO")
cursor = conn.cursor()

### Download data and load into pandas dataframe

In [2]:
sql = """SELECT 
DocumentReferenceContentAttachmentData, DocumentReferenceSubjectReference
FROM VectorSearchApp.DocumentReference"""

cursor.execute(sql)
out = cursor.fetchall()

import pandas as pd
## The result_set doesn't include column names so we will add them ourselvs
cols = ["ClinicalNotes", "Patient"] 

df = pd.DataFrame(out, columns=cols)

# Create a new column with PatientID as just a number
df["PatientID"] = pd.to_numeric(df["Patient"].astype(str).str.strip("Patient/"))

df.head()

,ClinicalNotes,Patient,PatientID
0,4f7469746973204d65646961204576616c756174696f6e...,Patient/4,4
1,446174653a20323032352d30382d30360a50726f766964...,Patient/4,4
2,466f6c6c6f772d557020666f72204f7469746973204d65...,Patient/4,4
3,446174653a20323032342d31312d32310a50726f766964...,Patient/4,4
4,446174653a20323032342d30382d30360a50726f766964...,Patient/4,4


### Decode the clinical notes into plain text

In [3]:
# This line uses a python lambda function (single-line function) to take a row of data 
# and decode the ClinicalNotes column, and return it to a new column called NotesDecoded 
df["NotesDecoded"] = df["ClinicalNotes"].apply(
    lambda x: bytes.fromhex(x).decode("utf-8", errors="replace")
    )
df.head()

,ClinicalNotes,Patient,PatientID,NotesDecoded
0,4f7469746973204d65646961204576616c756174696f6e...,Patient/4,4,Otitis Media Evaluation\nDate: 2024-01-25\nPro...
1,446174653a20323032352d30382d30360a50726f766964...,Patient/4,4,Date: 2025-08-06\nProvider: Dr. Jason347 Frami...
2,466f6c6c6f772d557020666f72204f7469746973204d65...,Patient/4,4,Follow-Up for Otitis Media\nDate: 2024-05-07\n...
3,446174653a20323032342d31312d32310a50726f766964...,Patient/4,4,Date: 2024-11-21\nProvider: Dr. Chin306 Kulas5...
4,446174653a20323032342d30382d30360a50726f766964...,Patient/4,4,Date: 2024-08-06\nProvider: Dr. Jason347 Frami...


In [4]:
## View one example of a note: 
print(df["NotesDecoded"][0])

Otitis Media Evaluation
Date: 2024-01-25
Provider: Dr. Lemuel304 Stokes453
Location: Beth Israel Deaconess Hospital – Needham
Reason for Visit: Ear pain and irritability
Subjective:
Aurora presented with symptoms of ear discomfort, mild fever, and increased fussiness. Parent reports onset 2 days ago. No vomiting or diarrhea. No prior history of ear infections.
Objective:

Vitals: Temp 37.8°C, BP 107/80 mmHg
Physical Exam:

Tympanic membrane: Erythematous and bulging on the right side
No discharge noted
Mild tenderness on palpation of the mastoid process
Lungs clear, no respiratory distress



Assessment:

Acute Otitis Media – Right ear

Plan:

Prescribed Amoxicillin 250 mg oral capsule, 1 capsule twice daily for 7 days
Supportive care: fluids, rest, acetaminophen for fever
Follow-up in 10 days or sooner if symptoms worsen
Educated parent on signs of complications


### Encoding clinical notes to vectors

Here we are using a pre-trained model from the hugging-faces `sentence_transformers` libary. Below, we use `all-MiniLM-L6-v2`. This is a basic sentence transformer model which outputs vectors with 384 dimensions. 

Before using the model, if you haven't already, installed sentence-transformers with pip.

There are many models available, some of which will be more relevant to your needs. For example, if you are using a language other than english, you would likely need a multi-lingual model or a model in your specific language. 

Before using the sentence-transformers library, you will need to install it if you haven't already. Uncomment and run the following install to do so. 

In [5]:
# pip install sentence-transformers

Note, this next step of loading the model and encoding the data into vectors will take a few minutes to run. 

In [6]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model. This model's output vectors are of size 384

model = SentenceTransformer('all-MiniLM-L6-v2') 

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['NotesDecoded'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['NotesVector'] = embeddings.tolist()

As we are using health data, an example of a better model for this process may be [pritamdeka/S-PubMedBert-MS-MARCO](https://huggingface.co/pritamdeka/S-PubMedBert-MS-MARCO), which is designed for medical text. This model has double the number of dimensions (768) as the model used above, which will slow the time taken to create the vectors. For this reason, this tutorial will stick to the `all-MiniLM-L6-v2`. 

If you do change the model used, remember to change the number of dimensions given when defining the SQL table below. 

In [7]:
## View output
df.head()

,ClinicalNotes,Patient,PatientID,NotesDecoded,NotesVector
0,4f7469746973204d65646961204576616c756174696f6e...,Patient/4,4,Otitis Media Evaluation\nDate: 2024-01-25\nPro...,"[0.037052784115076065, -0.01652533747255802, 0..."
1,446174653a20323032352d30382d30360a50726f766964...,Patient/4,4,Date: 2025-08-06\nProvider: Dr. Jason347 Frami...,"[-0.03362147510051727, 0.05015568435192108, 0...."
2,466f6c6c6f772d557020666f72204f7469746973204d65...,Patient/4,4,Follow-Up for Otitis Media\nDate: 2024-05-07\n...,"[-0.0073646255768835545, -0.02232077717781067,..."
3,446174653a20323032342d31312d32310a50726f766964...,Patient/4,4,Date: 2024-11-21\nProvider: Dr. Chin306 Kulas5...,"[-0.02497003600001335, 0.051254600286483765, 0..."
4,446174653a20323032342d30382d30360a50726f766964...,Patient/4,4,Date: 2024-08-06\nProvider: Dr. Jason347 Frami...,"[-0.006600956432521343, 0.041766293346881866, ..."


### Create new table in IRIS

Here we create a SQL query to create a new table in IRIS.

We have to define the datatypes required for this table. The vectors are loaded in the IRIS-specific data type 'VECTOR'. We also specify the type of each element in the vector (Double - a floating point number) and the dimensionality of the vector (384).


In [8]:
table_name = "VectorSearch.DocRefVectors"

create_table_query = f"""
CREATE TABLE {table_name} (
PatientID INTEGER,
ClinicalNotes LONGVARCHAR,
NotesVector VECTOR(DOUBLE, 384)
)
"""

In [9]:
cursor.execute(f"DROP TABLE IF EXISTS {table_name}" )
cursor.execute(create_table_query)

0

### Adding data

We will now insert our vector dataset into the table we've generated. There are multiple ways of adding data to our table, we could insert each row individually, iterating over each row individually. For this iteration, the pandas DataFrame method `df.apply()` can be used to efficiently perform the same function to each row of the data table. 

This can be speed up by executing many queries at once, which is shown below.

Importantly, the IRIS-SQL `TO_VECTOR()` function needs the vector to be in string format, so you will see both methods involve converting this data to a string before executing the query.

In [10]:
## Create a reusable query string with ? placeholders for the values
insert_query = f"INSERT INTO {table_name} ( PatientID, ClinicalNotes, NotesVector) values (?, ?, TO_VECTOR(?))"

In [11]:
## Needed for a speed comparison
import time

In [12]:
## Method 1: Inserting rows one at time (iterating with df.apply())
st = time.time()

def addRow(row): ## Create a function to insert each row
    cursor.execute(insert_query, 
                   [ row["PatientID"], 
                    row["NotesDecoded"], 
                    str(row["NotesVector"])])
    
## Apply the row insertion function to each row in the table (axis=1 specifies that we are iterating over rows, not columns). 
df.apply(addRow, axis=1)

print(f"Insertion took {time.time()-st} Seconds")

Insertion took 0.10890793800354004 Seconds


#### Optional: Speed up insertion

We can speed up the insertion by using the `cursor.executemany()` method, which uses the same insert query as above, but requires a list of parameter lists. 

In [13]:
## Reset the table between the insertion methods
cursor.execute(f"DROP TABLE IF EXISTS {table_name}" )
cursor.execute(create_table_query)

0

In [14]:
st = time.time()
df["NotesVectorStr"] = df["NotesVector"].astype(str)
rows_list = df[["PatientID", "NotesDecoded", "NotesVectorStr"]].values.tolist()


cursor.executemany(insert_query, rows_list)

print(f"Insertion took {time.time()-st} Seconds")

Insertion took 0.06851458549499512 Seconds


Both methods are quick because our table is very small. However, the second method is generally quicker. This speed boast might be useful if you are dealing with very large datasets. Either way though, we now have our data table complete with Vectors for use in a vector search! 

#### Advanced: Creating a specific index

For large databases, it may be possible to increase the insertion speed by separating the insertion and indexing. 

Unless you are struggling with slow insertion due to large datasets, this is unlikely to be necessary as the standard insertion is sufficiently efficient, so with small datasets, its not worth worrying about this enhancement. If you have datasets of >50,000 you may want to consider this extra step though. 

One of the reasons why InterSytems vector search is so efficient, particularly with large datasets, is due to the indexing of the vectors. When a row of vectors is inserted into a table containing a vector column it is automatically indexed. However automatic indexing can slow down the insertion of vectors into the database. 

A fix for this is to stop the default indexing when inserting vectors, and instead index the vectors after insertion. To do this, run the same query with the %NOINDEX keyword. Then, create an index using the syntax: 
```
INSERT %NOINDEX INTO <table-name> (Col1, Col2, Col3) (Val1, Val2, Val3);

CREATE INDEX <index-name> ON TABLE <table-name> (<vector-column-name>) AS HNSW(Distance='Cosine');
```

In [15]:
## Reset the table between the insertion methods
cursor.execute(f"DROP TABLE IF EXISTS {table_name}" )
cursor.execute(f"DROP INDEX IF EXISTS HNSWIndex" )
cursor.execute(create_table_query)

0

In [16]:
st = time.time()
#  
df["NotesVectorStr"] = df["NotesVector"].astype(str)
rows_list = df[["PatientID", "NotesDecoded", "NotesVectorStr"]].values.tolist()

## Create the insert query with %NOINDEX
insert_query = f"INSERT %NOINDEX INTO {table_name} ( PatientID, ClinicalNotes, NotesVector) values (?, ?, TO_VECTOR(?))"

## Execute the insert query for all our data
cursor.executemany(insert_query, rows_list)

## Create the index
cursor.execute(f"CREATE INDEX HNSWIndex ON TABLE {table_name} (NotesVector) AS HNSW(Distance='DotProduct')")

print(f"Insertion and index creation took {time.time()-st} Seconds")

Insertion and index creation took 0.18557167053222656 Seconds


So with our small dataset, it is actually slower to separate the inserting and indexing. However as the dataset gets larger, the time taken to index each row goes up dramatically, so it is better to separate these out.

### Querying table

Before moving on to the vector search, lets quickly query the database to check everything looks as expected:

In [17]:
sql_query  = f"Select TOP 3 * FROM {table_name}"
cursor.execute(sql_query) 
results = cursor.fetchall()
results_df = pd.DataFrame(results, columns= ["PatientID","NotesDecoded", "NotesVector"]  )
results_df.head()

,PatientID,NotesDecoded,NotesVector
0,4,Otitis Media Evaluation\nDate: 2024-01-25\nPro...,".037052784115076065063,-.016525337472558021546..."
1,4,Date: 2025-08-06\nProvider: Dr. Jason347 Frami...,"-.033621475100517272949,.050155684351921081542..."
2,4,Follow-Up for Otitis Media\nDate: 2024-05-07\n...,"-.0073646255768835544586,-.0223207771778106689..."


### Next Steps

Great, we've now got our database of clinical notes converted to Vector format based on semantic meaning, we can now continue by [implementing the vector search and prompting a Large Language model](./3-Vector-Search-LLM-Prompting.ipynb).